In [1]:
import pickle
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [19]:
with open("denoised_cv_results/models_denoising_accuracies.pkl", "rb") as f:
    models_denoising_accuracies = pickle.load(f)
with open("denoised_cv_results/models_denoising_f1_scores.pkl", "rb") as f:
    models_denoising_f1_score = pickle.load(f)

In [20]:
data = []

for model, denoise_dict in models_denoising_accuracies.items():
    for denoiser, accuracies in denoise_dict.items():
        for fold, acc in enumerate(accuracies):
            data.append({
                'model': model,
                'denoiser': denoiser,
                'fold': fold,
                'accuracy': acc
            })

df = pd.DataFrame(data)

anova_model = ols('accuracy ~ C(model) + C(denoiser) + C(model):C(denoiser)', data=df).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

print(anova_table)

                        sum_sq     df             F         PR(>F)
C(model)              0.516350    3.0   6223.460832  1.678917e-181
C(denoiser)           1.400575    4.0  12660.638235  2.437586e-219
C(model):C(denoiser)  0.371716   12.0   1120.054850  4.276723e-162
Residual              0.004978  180.0           NaN            NaN


In [21]:
df.groupby("denoiser")["accuracy"].mean()

from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=df["accuracy"], groups=df["denoiser"])
print(tukey)


       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1        group2    meandiff p-adj   lower   upper  reject
------------------------------------------------------------------
        DnCNN Gaussian_Blur  -0.1464    0.0 -0.1881 -0.1048   True
        DnCNN   Median_Blur  -0.1758    0.0 -0.2174 -0.1341   True
        DnCNN          None  -0.2388    0.0 -0.2804 -0.1971   True
        DnCNN     Restormer  -0.0666 0.0002 -0.1083 -0.0249   True
Gaussian_Blur   Median_Blur  -0.0293 0.3011  -0.071  0.0123  False
Gaussian_Blur          None  -0.0923    0.0  -0.134 -0.0506   True
Gaussian_Blur     Restormer   0.0799    0.0  0.0382  0.1215   True
  Median_Blur          None   -0.063 0.0004 -0.1047 -0.0213   True
  Median_Blur     Restormer   0.1092    0.0  0.0675  0.1508   True
         None     Restormer   0.1722    0.0  0.1305  0.2138   True
------------------------------------------------------------------


In [16]:
with open("masking_cv_results_v2/models_masks_accuracies.pkl", "rb") as f:
    models_masks_accuracies = pickle.load(f)
with open("masking_cv_results_v2/models_masks_f1_scores.pkl", "rb") as f:
    models_masks_f1_scores = pickle.load(f)

In [ ]:
data = []

for model, mask_dict in models_masks_accuracies.items():
    for mask, accuracies in mask_dict.items():
        for acc in accuracies:
            data.append({
                'model': model,
                'mask': mask,
                'accuracy': acc
            })

df = pd.DataFrame(data)

anova_model = ols('accuracy ~ C(model) + C(mask) + C(model):C(mask)', data=df).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

print(anova_table)

                    sum_sq     df           F        PR(>F)
C(model)          0.029189    3.0  480.562129  8.901237e-75
C(mask)           0.057555    3.0  947.576898  1.448025e-94
C(model):C(mask)  0.018118    9.0   99.429312  2.938280e-57
Residual          0.002915  144.0         NaN           NaN


In [ ]:
df.groupby("mask")["accuracy"].mean()

tukey = pairwise_tukeyhsd(endog=df["accuracy"], groups=df["mask"])
print(tukey)


    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
  group1     group2   meandiff p-adj   lower   upper  reject
------------------------------------------------------------
    Centre Non-centre   0.0048 0.6306 -0.0056  0.0152  False
    Centre       None   0.0332    0.0  0.0228  0.0436   True
    Centre     Random  -0.0199    0.0 -0.0303 -0.0095   True
Non-centre       None   0.0284    0.0   0.018  0.0388   True
Non-centre     Random  -0.0247    0.0 -0.0351 -0.0143   True
      None     Random  -0.0531    0.0 -0.0635 -0.0427   True
------------------------------------------------------------
